In [ ]:
remotes::install_github("jinworks/CellChat@88c2e13")

suppressPackageStartupMessages({
    library(CellChat)
    library(zellkonverter)
    library(SingleCellExperiment())
})



## Figure 3g and extended data figure 3d

In [ ]:
create_plots <- function(day) {
    cellchat <- readRDS(paste0("tmp/cellchat_",day,".rds"))
    # change the order of the labels
    cellchat@meta$labels <- forcats::fct_relevel(cellchat@meta$labels, "P14 top", "P14 crypt", "P14 muscularis")
    cellchat <- updateClusterLabels(cellchat, new.order = levels(cellchat@meta$labels))



    pdf(file = paste0("out/cc_day",day,".pdf"))
    plot(netAnalysis_signalingRole_heatmap(cellchat, pattern = "incoming"))
    plot.new()
    plot(netAnalysis_signalingRole_heatmap(cellchat, pattern = "outgoing"))
    dev.off()
}

In [ ]:
create_plots(day = 6)
create_plots(day = 8)
create_plots(day = 30)
create_plots(day = 90)

In [ ]:
get_in_and_out <- function(day) {
    object <- readRDS(paste0("tmp/cellchat_",day,".rds"))
    object@meta$labels <- forcats::fct_relevel(object@meta$labels, "P14 top", "P14 crypt", "P14 muscularis")
    object <- updateClusterLabels(object, new.order = levels(object@meta$labels))
    centr <- slot(object, "netP")$centr
    outgoing <- matrix(0, nrow = nlevels(object@idents), ncol = length(centr))
    incoming <- matrix(0, nrow = nlevels(object@idents), ncol = length(centr))
    dimnames(outgoing) <- list(levels(object@idents), names(centr))
    dimnames(incoming) <- dimnames(outgoing)
    for (i in 1:length(centr)) {
        outgoing[,i] <- centr[[i]]$outdeg
        incoming[,i] <- centr[[i]]$indeg
    }
    return(list(incoming=incoming, outgoing=outgoing))
}
res = lapply(X = c(6,8,30,90), get_in_and_out)
names(res) <- c(6,8,30,90)

In [ ]:
extract_values <- function(m, cells, pathways) {
    s = m|>
        as.data.frame() |>
        tibble::rownames_to_column("subset") |>
        tidyr::pivot_longer(-c(subset), names_to = "pathway")
    return(s)
}

df_plot = purrr::imap_dfr(res, \(x, y) {
    incoming = extract_values(x$incoming, cells, pathways)
    incoming$day = y
    incoming$direction = 'incoming'

    outgoing = extract_values(x$outgoing, cells, pathways)
    outgoing$day = y
    outgoing$direction = 'outgoing'
    return(rbind(incoming, outgoing))
})
df_plot |> head()

In [ ]:
m = df_plot |>
    dplyr::filter(subset %in% cells) |>
    dplyr::filter(direction == 'incoming') |>
    tidyr::pivot_wider(names_from = pathway, values_from = value)

cd = m[,1:2]
cd$day = factor(cd$day,levels = c(6,8,30,90))

m = m[,-c(1:3)] |> as.matrix() |> t()
m[is.na(m)] <- 0

In [ ]:
m <- sweep(m, 1L, apply(m, 1, max), '/', check.margin = FALSE)
m = na.omit(m)

In [ ]:
suppressPackageStartupMessages(library(ComplexHeatmap))

color.heatmap.use = grDevices::colorRampPalette((RColorBrewer::brewer.pal(n = 9, name = "BuGn")))(100)
pdf(file = "out/p14_cc_heatmap.pdf")
Heatmap(m,
col=color.heatmap.use,  
    cluster_rows = FALSE,
    cluster_columns = FALSE,
    column_split = cd$day,
    column_labels = cd$subset,
    border=TRUE,
    column_gap = unit(2, "mm"),
    rect_gp = gpar(col= "#FFFFFF"), 
    name="incoming signals"
)
dev.off()

## Extended data figure 4a

In [ ]:
sce = readH5AD("tmp/adata_gated.h5ad")

In [ ]:
# Define colors
color.use = colors = c(
    "Epithelial_Secretory" = "#AA9228",
    "Epithelial_Absorptive" = "#E3C300",
    "Monocyte" = "#C37698",
    "T-Cell" = "#008E74",
    "MAIT" = "#63ABB9",
    "Myeloid" = "#EF9684",
    "ILC" = "#A0C6D3",
    "B-Cell" = "#E2CEAB",
    "DC" = "#FE757D",
    "Fibroblast" = "#E17300",
    "Endothelial" = "#E30133",
    "NK" = "#4A7B89",
    "Epithelial_Progenitor" = "#F7BC00",
    "Neuron" = "#2A2446",
    "Erythroid" = "#A5021D",
    "Eosinophil" = "#782c4e"
)

subtype_to_type <- colData(sce)[c('Subtype_gate', 'Type')] |>
    as.data.frame() |>
    dplyr::distinct()

color.use <- color.use |> as.data.frame() |> tibble::rownames_to_column("Type")

subtype_to_type <- subtype_to_type |> left_join(color.use)



In [ ]:
cellchat <- readRDS(paste0("tmp/cellchat_full.rds"))
pathways.show <- c("TGFb") 

idx <- match(levels(cellchat@meta$labels), subtype_to_type$Subtype_gate)
color.use <- subtype_to_type$color.use[idx]
names(color.use) <- subtype_to_type$Subtype_gate

vertex.receiver	<- which(levels(cellchat@idents) %in% c("P14 top", "P14 crypt", "P14 muscularis"))

netVisual_individual(cellchat, signaling = pathways.show, layout = 'circle', targets.use=vertex.receiver, color.use=color.use)
